In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import lxml
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics
from sklearn.metrics import mean_squared_error,r2_score
import tensorflow as tf
import pickle

from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD 
from tensorflow.keras.callbacks import EarlyStopping

import itertools
from tensorflow.keras.layers import LSTM

from tensorflow.keras.layers import Dropout

res=pd.read_csv(r"C:\Users\aksha\Desktop\Result.csv")

Y=res[['Space_heating', 'Appliances',
       'Lighting']]
X=res[['Minimum Temperature', 'Maximum Temperature', 'Temperature',
       'Dew Point', 'Relative Humidity', 'Global_active_power',
       'Global_intensity']]


from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split



# normalize the data
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
y = scaler.fit_transform(Y)

# split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# reshape the data into 3D format
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# define the model
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(LSTM(50))
model.add(Dense(3))
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])

# fit the model to the training data
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

# evaluate the model on the test data
y_pred = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test)
mse = mean_squared_error(y_test, y_pred)

# save the model
model.save("time_series_model.h5")

objects=[model,scaler]
with open("objects.pkl", "wb") as f:
    pickle.dump(objects, f)